# Building a Multi-Page FastHTML Site with Live Notebook Refresh

Let's build a simple multi-page website with FastHTML. We'll start with a home page that has a navigation button to an about page.

First, we create the necessary imports and boilerplate:

In [226]:
from dataclasses import dataclass, field
from functools import wraps
from typing import Callable, Any, List, Dict
from fasthtml.common import *
from fasthtml.jupyter import *
from IPython.display import display, HTML 

def requires_shell(func: Callable) -> Callable[..., Optional[Any]]:
    """Decorator to ensure shell is available"""
    @wraps(func)
    def wrapper(self: NotebookContext, *args, **kwargs) -> Optional[Any]:
        return None if not self._shell else func(self, *args, **kwargs)
    return wrapper

@dataclass
class NotebookContext:
    """Manages IPython/Jupyter notebook context and variable tracking"""
    pages: List['Page'] = field(default_factory=list)
    _shell: Any = field(init=False, default=None)
    
    def __post_init__(self) -> None:
        """Initialize IPython shell connection"""
        from IPython import get_ipython
        self._shell = get_ipython()    
    
    def register_page(self, page: 'Page') -> None:
        """Register a page for variable tracking"""
        self.pages.append(page)
    
    @requires_shell
    def auto_update_cell(self, _=None) -> None:
        """Update tracked variables that changed in last cell execution"""
        for page in self.pages:
            self._check_page_variables(page)
    
    def _check_page_variables(self, page: 'Page') -> None:
        """Check for changes in a page's tracked variables"""
        # Look for refs of type 'var'
        var_refs = {
            name: ref['elem'] 
            for name, ref in page.refs.items() 
            if ref['type'] == 'var'
        }
        
        for var_name in list(var_refs.keys()):
            if var_name not in self._shell.user_ns:
                continue
                
            new_value = self._shell.user_ns[var_name]
            old_value = var_refs[var_name]
            
            if new_value is not old_value:
                page.update_element(new_value, var_name)
    
    @requires_shell
    def get_variable_name(self, value: Any) -> str | None:
        """Find variable name in notebook namespace"""
        user_vars = {
            name: val 
            for name, val in self._shell.user_ns.items() 
            if not name.startswith('_')
        }
        
        return next(
            (name for name, val in user_vars.items() if val is value), 
            None
        )
    
    @requires_shell
    def register_callback(self, callback: Callable) -> None:
        """Register a post-cell execution callback"""
        self._shell.events.register('post_run_cell', callback)        
    
    @requires_shell
    def unregister_callback(self, callback: Callable) -> None:
        """Unregister a post-cell execution callback"""
        self._shell.events.unregister('post_run_cell', callback)        
    
@dataclass
class Page:
    client: Any 
    context: NotebookContext
    name: str = ""
    elements: List = field(default_factory=list)
    refs: Dict[str, Dict[str, Any]] = field(default_factory=dict)
    active: bool = True
    dirty: bool = False

    def _update_element_at(self, elem: Any, idx: int) -> None:
        """Update element at specific index and its references"""
        self.elements[idx] = elem
        
        # Update any existing refs pointing to this index
        for key, ref in list(self.refs.items()):
            if ref['index'] == idx:
                ref['elem'] = elem

    def _handle_element(self, elem: Any) -> None:
        """Add or update a single element"""
        elem_id = getattr(elem, 'id', None)
        var_name = self.context.get_variable_name(elem)
        
        # Find existing index from refs
        idx = next(
            (ref['index'] 
             for k in [elem_id, var_name] 
             if k and k in self.refs 
             and (ref := self.refs[k])),
            None
        )
        
        if idx is not None:
            self._update_element_at(elem, idx)
        else:
            idx = len(self.elements)
            self.elements.append(elem)
                    
        for key, type_ in [(var_name, 'var'), (elem_id, 'id')]:
            if key:
                self.refs[key] = {'type': type_, 'index': idx, 'elem': elem}
        
        self.dirty = True

    def add(self, *elements: Any) -> 'Page':
        """Add or update elements"""
        for elem in elements:
            self._handle_element(elem)
        return self.update()

    def update_element(self, elem: Any, var_name: str | None = None) -> None:
        """Update or remove element by variable name"""
        if var_name is None:
            var_name = self.context.get_variable_name(elem)
        
        if not var_name:
            return
            
        if elem is None:
            self.remove(var_name)
        else:
            self._handle_element(elem)
            self.update()

    def remove(self, *ref_keys: str) -> 'Page':
        """Remove elements by reference keys"""
        removed_indices = []
        
        for key in ref_keys:
            if ref := self.refs.get(key):
                removed_indices.append(ref['index'])
                del self.refs[key]
        
        # Remove elements in reverse order to maintain correct indices
        for idx in sorted(removed_indices, reverse=True):
            self.elements.pop(idx)
            # Update remaining refs
            for ref in self.refs.values():
                if ref['index'] > idx:
                    ref['index'] -= 1
        
        if removed_indices:
            self.dirty = True
            
        return self.update()

    def update(self) -> 'Page':
        """Update page content if active and dirty"""
        if self.active and self.dirty:
            self.client.set(*self.elements)
            self.dirty = False
        return self

    def clear(self) -> 'Page':
        """Clear all elements from the page"""
        self.elements.clear()
        self.refs.clear()
        self.dirty = True
        return self.update()
    
    def get_html(self) -> str:
        """Get HTML representation of current page"""
        if not self.elements:
            return ""
        from fasthtml.common import to_xml
        html = "\n".join(to_xml(elem) for elem in self.elements)
        print(html)

    def __repr__(self) -> str:
        route_name = self.name or "home"
        url = f"http://localhost:8000/{self.name}"
        return f"Page(route='{route_name}', elements={len(self.elements)}, url='{url}')"
    
@dataclass
class PageManager:
    """Manages FastHTML pages and websocket connections"""
    exts: str = "ws"
    pages: Dict[str, Page] = field(default_factory=dict)
    _context: NotebookContext = field(init=False)
    _app: Any = field(init=False)
    _server: Any = field(init=False)
    _callback_registered: bool = field(init=False, default=False)
    
    def __post_init__(self) -> None:
        """Initialize FastHTML app and notebook context"""
        # Setup FastHTML
        self._app = FastHTML(exts=self.exts)
        self._server = JupyUvi(self._app)
        setup_ws(self._app)
        
        # Setup notebook context
        self._context = NotebookContext()
        self._callback_registered = self._context.register_callback(
            self._context.auto_update_cell
        )
    
    def create_page(self, route: str = "", frame: bool = False) -> Page:
        """Create a new page at the specified route"""
        client = ws_client(self._app, route, frame=frame, link=False)
        
        # Create and register page
        page = Page(client, self._context, route)
        self.pages[route] = page
        self._context.register_page(page)
        
        # Show page link
        self._display_page_link(route)
        return page
    
    def _display_page_link(self, route: str) -> None:
        """Display clickable link to page"""
        route_name = route or "home"
        url = f"http://localhost:8000/{route}"
        display(HTML(f'<a href="{url}" target="_blank">View {route_name} page</a>'))
    
    def update_all(self) -> None:
        """Update all active pages that have changes"""
        for page in self.pages.values():
            if page.active and page.dirty:
                page.update()
    
    def stop(self) -> None:
        """Clean up manager resources"""
        if self._callback_registered:
            self._context.unregister_callback(self._context.auto_update_cell)
            self._callback_registered = False
        if hasattr(self, '_server'):
            self._server.stop()
    
    def __del__(self) -> None:
        """Ensure cleanup on deletion"""
        self.stop()
    
    def activate_route(self, route: str) -> None:
        """Enable auto-updates for a route"""
        if page := self.pages.get(route):
            page.active = True
    
    def deactivate_route(self, route: str) -> None:
        """Disable auto-updates for a route"""
        if page := self.pages.get(route):
            page.active = False

## Setting Up Our Pages

The `PageManager` class handles:
- Creating and managing multiple pages
- Automatic updates when we modify content
- WebSocket connections for live refresh
- Server cleanup when we're done

When we create a page with `create_page()`:
- An empty route `""` creates the index/home page at `localhost:8000/`
- Any other route like `"about"` creates a page at `localhost:8000/about`

Let's create our home page and add some initial content:

In [227]:
manager = PageManager()

home = manager.create_page("") # index route

home.add(
    H1("Welcome!"),
    P("This is a multi-page FastHTML site"),
    Button("Learn More About Me")
)

Page(route='home', elements=3, url='http://localhost:8000/')

We get a link to view the page in the browser when creating a page. As well a custom string representation of the page that shows the route and number of elements.

The button currently doesn't do anything. Let's fix that. 🤗

## Adding Navigation Between Pages

let's create an about page and add navigation between our pages. We'll:
1. Create a new page for the about route
2. Update our home page button content to include navigation
3. Add content to the about page with a return link back to home

Note: When using `clear().add()`, we're explicitly removing all content before adding new elements.

In [228]:
about = manager.create_page("about")

# Update home page with navigation
home.clear().add(
    H1("Welcome!"),
    P("This is a multi-page FastHTML site"),
    Button("Learn More About Me", hx_get="/about", hx_target="body")
)

# Add content to about page
about.add(
    H1("About Me"),
    P("I enjoy using FastHTML and HTMX."),
    Button("Back to Home", hx_get="/", hx_target="body")
)

Page(route='about', elements=3, url='http://localhost:8000/about')

## Dynamic Content Updates

FastHTML with PageManager supports two approaches for live content updates:

1. **Variable Tracking**: Keep a reference to an element and update it
2. **ID-Based Updates**: Add elements with IDs and update them by referencing the same ID

Both methods will automatically refresh the page content without needing a browser reload.

In [229]:
# Approach 1: Variable tracking
content = P("Initial content")
about.add(content)

Page(route='about', elements=4, url='http://localhost:8000/about')

In [230]:
# Update the variable:
# our PageManager will automatically track and update elements assigned to variables
content = P("Content updated via variable tracking!")

In [231]:
# Approach 2: ID-based updates
# First, add elements with unique IDs
about.add(
    P("Initial content with ID", id="dynamic-content"),
    Button("Click me!", id="action-button")
)

Page(route='about', elements=6, url='http://localhost:8000/about')

In [232]:
# Update element by referencing the same ID
# This will replace the existing element while maintaining its position
about.add(
    P("Updated content with ID", id="dynamic-content")
)

Page(route='about', elements=6, url='http://localhost:8000/about')

In [223]:
# Setting a tracked variable to None or "" removes it from the page
content = None

In [233]:
# Remove elements by their ID
about.remove("dynamic-content")

Page(route='about', elements=5, url='http://localhost:8000/about')

In [234]:
# Get current HTML representation of the page
# Useful for debugging or verification
about.get_html()

<h1>About Me</h1>

<p>I enjoy using FastHTML and HTMX.</p>

<button hx-get="/" hx-target="body">Back to Home</button>
<p>Content updated via variable tracking!</p>

<button id="action-button" name="action-button">Click me!</button>


In [240]:
# Summary: Dynamic Content Updates
# 1. Variable tracking
content = P("Initial content")
about.add(content)
content = P("Updated content - watch it change!")  # Auto-updates!

# 2. ID-based updates
about.add(
    P("Initial content with ID", id="dynamic-content"),
    Button("Click me!", id="action-button")
)
about.add(P("Updated content with ID", id="dynamic-content"))

Page(route='about', elements=6, url='http://localhost:8000/about')

In [244]:
# Example: Controlling Page Updates
# Stop auto-updates for about page
# manager.deactivate_route("about")  
# content = P("This won't auto-update")
# about.add(content)

# Re-enable auto-updates
# manager.activate_route("about")    
# content = P("This will auto-update again!")

In [225]:
# Cleanup of port/resources
manager.stop()

1. **Auto-Updates**: Pages automatically update when tracked variables change. You can control this with:
   - `manager.deactivate_route()` to pause updates
   - `manager.activate_route()` to resume updates

2. **Cleanup**: Always call `manager.stop()` when you're done to:
   - Free the port (8000)
   - Unregister notebook callbacks
   - Release server resources

3. **Best Practices**:
   - Use variable tracking for simple updates
   - Use ID-based updates for more complex element management
   - Keep pages active unless you specifically need to pause updates